# IPv4

<img src="res/ip01.png" width="500px">
<img src="res/ip02.png" width="500px">
<img src="res/ip03.png" width="500px">
<img src="res/ip04.png" width="500px">
<img src="res/ip05.png" width="500px">

1. IP fragment/defragment 本身有一些问题，比如乱序，id重复，丢失等等。这些问题ip层面是解决不了的，只能依赖上层协议（比如tcp）

<img src='res/ip06.png' width=500px>

## linux2.6/net/ipv4/ip_fragment.c

```c
/* Describe an entry in the "incomplete datagrams" queue. */
struct ipq {
	struct ipq	*next;		/* linked list pointers			*/
	struct list_head lru_list;	/* lru list member 			*/
	u32		user;
	u32		saddr;
	u32		daddr;
	u16		id;
	u8		protocol;
	u8		last_in;
#define COMPLETE		4
#define FIRST_IN		2
#define LAST_IN			1

	struct sk_buff	*fragments;	/* linked list of received fragments	*/
	int		len;		/* total length of original datagram	*/
	int		meat;
	spinlock_t	lock;
	atomic_t	refcnt;
	struct timer_list timer;	/* when will this queue expire?		*/
	struct ipq	**pprev;
	int		iif;
	struct timeval	stamp;
};
```

1. 这是一个双向链表的节点。有意思的点在于其 prev 的指针是一个指针的指针，存储的是prev节点中的next指针的地址。这个**实现很巧妙**

2. 每个queue的头需要存在hash table中。如果hash table直接存放节点struct，每次增删，rehash，代价很大。所以只能存放节点指针

3. 如果pprev不用 double 指针，那么每次操作比如删除，都要判断下当前是不是队列头，要对hash表中的位置进行单独操作。而用 double pointer，无论操作队列头还是中间节点，都是一样的

4. 这其实是kernel中hash link的统一的数据结构

```c
// hash 桶中存放的指针类型
struct hlist_head {
	struct hlist_node *first;
};

// 队列的指针类型
struct hlist_node {
	struct hlist_node *next;
	struct hlist_node **pprev;
};
```
5. 具体可以参看 [kernel hlist](res/ip07.png)

[kernel hlist详解](https://blog.csdn.net/hs794502825/article/details/24597773)

-----------------------